In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [2]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [3]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex"
today

'day24'

In [4]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [5]:
with open(f"./data/{today}.txt") as f:
    lines = f.read().rstrip()

raw_dependencies: list[tuple[str, str, str, str]] = []  # (left, right, operator, target)
dependencies_init: dict[str, tuple[str, str, str]] = {}  # target -> (left, right, operator)
downstream_init: dict[str, list[str]] = defaultdict(list)
for line in lines.split("\n\n")[1].rstrip().split("\n"):
    first = line[:3]
    second = line.split(" -> ")[0][-3:]
    third = line[-3:]
    operator = line.split(" -> ")[0][4:-4]
    assert operator in ("AND", "OR", "XOR"), operator

    raw_dependencies.append((first, second, operator, third))
    dependencies_init[third] = (first, second, operator)
    downstream_init[first].append(third)
    downstream_init[second].append(third)

In [6]:
def run_graph(
    init: set[str],
    dependencies: dict[str, tuple[str, str, str]],
    downstream: dict[str, list[str]],
) -> str:
    activated = deque()
    values: dict[str, bool] = {}
    for i in range(45):
        key = "x" + str(i).rjust(2, "0")
        activated.append(key)
        values[key] = key in init
        
        key = "y" + str(i).rjust(2, "0")
        activated.append(key)
        values[key] = key in init

    while len(activated) > 0:
        wire = activated.pop()
        for next_wire in downstream[wire]:
            if next_wire in values:
                continue
            left, right, op = dependencies[next_wire]

            if left not in values or right not in values:
                continue
            activated.appendleft(next_wire)
            if op == "AND":
                values[next_wire] = values[left] and values[right]
            elif op == "OR":
                values[next_wire] = values[left] or values[right]
            else:
                values[next_wire] = values[left] ^ values[right]
    
    z_wires = [(l, v) for (l, v) in values.items() if l[0] == "z"]    
    z_wires = reversed(sorted(z_wires, key=lambda x: x[0]))
    z_wire_values = [("1" if v else "0") for (_, v) in z_wires]
    return "".join(z_wire_values)

In [7]:
init = set()
for line in lines.split("\n\n")[0].split("\n"):
    if line.rstrip()[-1] != "1":
        continue
    wire = line.split(": ")[0]
    init.add(wire)

int(run_graph(init, dependencies_init, downstream_init), 2)

45923082839246

In [8]:
def diff(s1: str, s2: str) -> int:
    for i, (c1, c2) in enumerate(zip(reversed(s1), reversed(s2))):
        if c1 != c2:
            return i
    return 999

def score(
    test_nums: list[tuple[int, int]],
    dependencies: dict[str, tuple[str, str, str]],
    downstream: dict[str, list[str]],
    limit: int,
) -> int:
    res = 999
    for i in range(45):
        output = run_graph({"x" + str(i).rjust(2, "0")}, dependencies, downstream)
        expected = "0" * (45 - i) + "1" + "0" * i
        d = diff(output, expected)
        res = min(res, d)
        if d <= limit:
            return d
    for i in range(45):
        output = run_graph({"y" + str(i).rjust(2, "0")}, dependencies, downstream)
        expected = "0" * (45 - i) + "1" + "0" * i
        d = diff(output, expected)
        res = min(res, d)
        if d <= limit:
            return d
    for i in range(45):
        output = run_graph({"x" + str(i).rjust(2, "0"), "y" + str(i).rjust(2, "0")}, dependencies, downstream)
        expected = "0" * (45 - i - 1) + "1" + "0" * (i + 1)
        d = diff(output, expected)
        res = min(res, d)
        if d <= limit:
            return d

    for x, y in test_nums:
        init = set()
        for i in range(45):
            if (x >> i) % 2:
                init.add("x" + str(i).rjust(2, "0"))
            if (y >> i) % 2:
                init.add("y" + str(i).rjust(2, "0"))
        output = run_graph(init, dependencies, downstream)
        expected = bin(x + y)[2:]
        d = diff(output, expected)
        res = min(res, d)
        if d <= limit:
            return d

    return res

In [9]:
def swap(
    ind1: int,
    ind2: int,
    raw_dependencies: list[tuple[str, str, str, str]],
    dependencies: dict[str, tuple[str, str, str]],
    downstream: dict[str, list[str]],
    back: bool = False,
) -> None:
    first1, second1, op1, third1 = raw_dependencies[ind1]
    first2, second2, op2, third2 = raw_dependencies[ind2]

    if not back:
        dependencies[third1] = (first2, second2, op2)
        dependencies[third2] = (first1, second1, op1)

        downstream[first1].remove(third1)
        downstream[first1].append(third2)
        downstream[second1].remove(third1)
        downstream[second1].append(third2)

        downstream[first2].remove(third2)
        downstream[first2].append(third1)
        downstream[second2].remove(third2)
        downstream[second2].append(third1)
        return
    
    dependencies[third1] = (first1, second1, op1)
    dependencies[third2] = (first2, second2, op2)

    downstream[first1].remove(third2)
    downstream[first1].append(third1)
    downstream[second1].remove(third2)
    downstream[second1].append(third1)

    downstream[first2].remove(third1)
    downstream[first2].append(third2)
    downstream[second2].remove(third1)
    downstream[second2].append(third2)

In [10]:
import random

test_nums = []
for x in random.sample(range(2 << 44), 50):
    for y in random.sample(range(2 << 44), 50):
        test_nums.append((x, y))

In [11]:
dependencies = deepcopy(dependencies_init)
downstream = deepcopy(downstream_init)

swapped = set()
curr_score = score(test_nums, dependencies, downstream, 0)
print(curr_score)

9


In [12]:
for _ in range(4):
    for i in range(len(raw_dependencies)):
        flag = False
        if i % 10 == 0:
            print(f"{i} / {len(raw_dependencies)}")
        if i in swapped:
            continue
        for j in range(i+1, len(raw_dependencies)):
            if j in swapped:
                continue
            swap(i, j, raw_dependencies, dependencies, downstream)
            s = score(test_nums, dependencies, downstream, curr_score)
            if s > curr_score:
                curr_score = s
                swapped.add(i)
                swapped.add(j)
                print(f"Proposed swap: {(i, j)} -> new score {curr_score}")
                flag = True
                break
            swap(i, j, raw_dependencies, dependencies, downstream, True)
        if flag:
            break

0 / 222
10 / 222
20 / 222
30 / 222
40 / 222
50 / 222
60 / 222
70 / 222
80 / 222
90 / 222
100 / 222
110 / 222
120 / 222
130 / 222
140 / 222
150 / 222
160 / 222
170 / 222
180 / 222
Proposed swap: (181, 209) -> new score 20
0 / 222
10 / 222
20 / 222
30 / 222
40 / 222
50 / 222
60 / 222
Proposed swap: (60, 66) -> new score 24
0 / 222
10 / 222
Proposed swap: (13, 80) -> new score 31
0 / 222
10 / 222
20 / 222
30 / 222
40 / 222
50 / 222
60 / 222
70 / 222
80 / 222
90 / 222
100 / 222
110 / 222
120 / 222
130 / 222
Proposed swap: (135, 186) -> new score 999


In [13]:
print(",".join(sorted([raw_dependencies[ind][-1] for ind in swapped])))

jgb,rkf,rrs,rvc,vcg,z09,z20,z24
